In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip install --no-deps trl peft accelerate bitsandbytes
!pip install xformers transformers datasets torch trl

!pip install ctransformers langchain streamlit

In [ ]:
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
from datasets import load_from_disk
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer


# dataset = load_dataset("lighteval/med_dialog",'')
dataset = load_dataset('lighteval/med_dialog', 'healthcaremagic')

dataset = dataset.remove_columns(['tgt','id'])



def generate_prompt(user:str , system:str)->str:
    return f"""
        Below is an instruction that describes a task, paired with an input that provides further context.
        Write a response that appropriately completes the request.

        Instruction -
        You are an AI medical assistant.Your role is to engage in a thoughtful
        dialogue with user to fully understand  symptoms and health concerns.

        user query -
        {user}

        your response -
        {system}
     """

def parse_conversation_to_df(text):
    text = text['src']
    data = {'prompt':""}
    messages = text.split("Patient: ")[1:]
    # messages = ' '.join(**messages.strip().split())

    # print(messages)
    for msg in messages:
        msg = ' '.join(msg.strip().split())

        # Splitting the message by "Doctor: "
        parts = msg.rsplit("Doctor: ", 1)
        # Extracting patient's message
        patient_msg = parts[0].strip()
        # Extracting doctor's message
        doctor_msg = parts[1].strip() if len(parts) > 1 else ""
        # Removing unnecessary spaces
        patient_msg = ' '.join(patient_msg.split())
        doctor_msg = ' '.join(doctor_msg.split())
        # Appending messages to the data dictionary
        data["prompt"]=generate_prompt(system=doctor_msg.strip(), user=patient_msg.strip())
    # df = pd.DataFrame(data)
    return data


print(dataset)
from datasets import concatenate_datasets

# Combine train and test datasets
combined_dataset = concatenate_datasets([dataset['train'], dataset['test']])


dataset = concatenate_datasets([combined_dataset, dataset['validation']])



dataset = dataset.map(parse_conversation_to_df).remove_columns(['src']).with_format('pt')

# print(dataset['train'][0])



max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth


model_name = "unsloth/llama-3-8b-bnb-4bit"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,

    dtype = None,
    load_in_4bit = True,
)


# Do model patching and add fast LoRA weights
model1 = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



trainer = SFTTrainer(
    model = model1,
    train_dataset = dataset,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        learning_rate = 2e-4,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

model1.save_pretrained(model_name + "_lora_model1")

# Go to https://github.com/unslothai/unsloth/wiki for advanced tips like
# (1) Saving to GGUF / merging to 16bit for vLLM
# (2) Continued training from a saved LoRA adapter
# (3) Adding an evaluation loop / OOMs
# (4) Cutomized chat templates

# Gpu 1 - full utilization
# vram nearly 7gb
# temperature 60 to 70 C
# power >220W

max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



from unsloth import FastLanguageModel

model_tuned, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name + "_lora_model1", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,



)
FastLanguageModel.for_inference(model_tuned) # Enable native 2x faster inference


from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


def summarize(model, tokenizer ,  text: str):
    inputs = tokenizer(text, return_tensors="pt")
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


# summary = summarize(model, 'I am sweating a lot these days and feeling tired.')
# print('Before Fine tuning -  ',summary)
import time

start = time.time()

summary = summarize(model=model_tuned, tokenizer=tokenizer, text="""
        You are an AI medical assistant, your role is to engage in a thoughtful
        dialogue with user to fully understand  symptoms and health concerns.
        Ask  questions one at a time to gather relevant information.
        You will  ask follow-up questions to clarify and gather more details
        about the problem. After your conversation, you will provide a summary,
        including any recommendations for foods to avoid, medications needed,
        an explanation of the potential condition, and suggestions for further
        action. Based on responses, you will aim to identify potential causes,
        provide advice on symptoms/diagnosis, suggest foods to avoid, recommend
        medications or lifestyle changes,
        and indicate when it may be necessary to consult a doctor.

        user query -
        I am sweating a lot these days and feeling tired.

        Your Response -""")

summary_before = summarize(model=model, tokenizer=tokenizer, text="""
        You are an AI medical assistant, your role is to engage in a thoughtful
        dialogue with user to fully understand  symptoms and health concerns.
        Ask  questions one at a time to gather relevant information.
        You will  ask follow-up questions to clarify and gather more details
        about the problem. After your conversation, you will provide a summary,
        including any recommendations for foods to avoid, medications needed,
        an explanation of the potential condition, and suggestions for further
        action. Based on responses, you will aim to identify potential causes,
        provide advice on symptoms/diagnosis, suggest foods to avoid, recommend
        medications or lifestyle changes,
        and indicate when it may be necessary to consult a doctor.

        user query -
        I am sweating a lot these days and feeling tired.

        Your Response -""")


end = time.time()

print('Before Fine tuning - ',summary_before)

print(end-start)

end = time.time()

print('After Fine tuning - ',summary)

print(end-start)






🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Generating train split:   0%|          | 0/181122 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/22641 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22642 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['src'],
        num_rows: 181122
    })
    validation: Dataset({
        features: ['src'],
        num_rows: 22641
    })
    test: Dataset({
        features: ['src'],
        num_rows: 22642
    })
})


...

Map:   0%|          | 0/226405 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 226,405 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.013300
2,2.911100
3,3.041800
4,3.043400
5,3.044500
6,2.794700
7,2.722900
8,2.439900
9,2.339400
10,2.352500


==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
